In [1]:
from qiskit import Aer, transpile, assemble, execute
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
import numpy as np


To create a software implementation of the algorithm based on Hamiltonian simulation and phase estimation for eigendecomposition and then inverting the eigenvalues, here is a small example with a 4x4 matrix with a step-by-step breakdown.

### Step 1: Problem Definition
You want to solve a system of equations \( A \vec{x} = \vec{b} \), where \( A \) is a 4x4 matrix.

### Step 2: Hamiltonian Simulation and Phase Estimation

Hamiltonian simulation is about evolving a quantum state according to a Hamiltonian \( H \), which is derived from the matrix \( A \). Since \( A \) is 4x4, we can represent it as a 2-qubit system. Our goal is to create a unitary \( U \) such that \( U|0\rangle = e^{iAt}|b\rangle \).


In [2]:
def hamiltonian_simulation(qc, qubit, theta):
    qc.rz(theta, qubit)


The Quantum Phase Estimation (QPE) algorithm estimates the phase (or eigenvalues) of a unitary operator. After applying QPE on \( U \), the output will be an approximation to the eigenvalues of \( A \).


In [3]:
def qpe(matrix):
    # Number of qubits for the eigenvalue in the output
    num_qubits = 4 
    qr = QuantumRegister(num_qubits + 1, 'q')
    cr = ClassicalRegister(num_qubits, 'c')
    qc = QuantumCircuit(qr, cr)

    # Prepare the eigenstate |1⟩. 
    qc.x(qr[0])

    # Apply Hadamard to ancilla qubits
    for qubit in range(1, num_qubits + 1):
        qc.h(qubit)

    # Controlled-Hamiltonian simulation
    angle = np.pi / 4  
    repetitions = 1
    for counting_qubit in range(1, num_qubits + 1):
        for _ in range(repetitions):
            hamiltonian_simulation(qc, counting_qubit, angle)
        repetitions *= 2

    qc.append(QFT(num_qubits).inverse(), qr[1:num_qubits + 1])
    qc.measure(qr[1:], cr)
    
    # Run the quantum circuit on a simulator backend
    aer_sim = Aer.get_backend('aer_simulator')
    # Transpile the circuit for the selected backend
    t_qc = transpile(qc, aer_sim)
    # Create a Qobj from the circuit for the simulator to run
    qobj = assemble(t_qc)
    # Run the simulator
    result = aer_sim.run(qobj).result()
    # Get the results
    counts = result.get_counts(qc)
    
    return counts


### Step 3: Inversion of the Eigenvalues

Once we have the eigenvalues from the phase estimation, we can invert them. This is a classical computation step. If the eigenvalue is \( \lambda \), its inverse will be \( \frac{1}{\lambda} \).

In [4]:
def invert_eigenvalues(counts):
    # Get the most common eigenvalue
    most_common = sorted(counts.items(), key=lambda x: x[1], reverse=True)[0][0]
    # Convert to fraction
    eigenvalue = int(most_common, 2) / 2**len(most_common)
    # Invert the eigenvalue
    if eigenvalue != 0:
        inverted_value = 1 / eigenvalue
    else:
        inverted_value = float('inf')  # Handling division by zero
    return inverted_value


### Step 4: Postselection

Instead of amplitude amplification, we will use postselection. This means we will run our quantum algorithm many times and simply choose the output that gives us the desired accuracy. In a real-world scenario, amplitude amplification would be a more efficient choice, but postselection simplifies our implementation.

In [5]:
if __name__ == "__main__":
    matrix = [[1, 0], [0, 1]]  # Example identity matrix for simplicity
    counts = qpe(matrix)
    inverted_value = invert_eigenvalues(counts)
    print("Inverted Eigenvalue:", inverted_value)
    plot_histogram(counts).show()


Inverted Eigenvalue: 8.0


/var/folders/my/3k6ljkkn2fxf0bmxsr7jkrf00000gp/T/ipykernel_20537/1835944970.py:6: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plot_histogram(counts).show()


In [6]:
plot_histogram(counts).savefig("histogram.png")
